In [ ]:
from osm_ai_helper.download_osm import download_osm

In [ ]:
AREA = "Ponteareas"

download_osm(
    area=AREA,
    output_dir="dataset",
    selector="leisure=swimming_pool",
    discard={"location": "indoor"},
)

In [ ]:
from osm_ai_helper.group_elements_and_download_tiles import (
    group_elements_and_download_tiles,
)

In [ ]:
import os

os.environ["MAPBOX_TOKEN"] = (
    ""
)

In [ ]:
group_elements_and_download_tiles(
    "dataset/Ponteareas.json",
    "dataset/Ponteareas",
)

In [ ]:
from osm_ai_helper.convert_to_vlm_dataset import convert_to_vlm_dataset

In [ ]:
dataset = convert_to_vlm_dataset(
    "dataset/Ponteareas", "Point to the swimming pools in the image."
)

In [ ]:
from osm_ai_helper.utils.plots import show_vlm_entry

In [ ]:
show_vlm_entry(dataset[6])